# 패키지 로드

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time
import requests
import csv

# 리뷰 수집

함수설정

In [2]:
def page_count(num, per_page):
    if num % per_page ==0:
        page = int(num / per_page)
    else:
        page = int(num / per_page) + 1
    return page

In [3]:
def review_count():
    review_num = driver.find_elements(By.CLASS_NAME, f'_2pgHN-ntx6')[0].text
    if ',' in review_num:
        review_num = int(review_num.replace(',', ''))
    else:
        review_num = int(review_num)
    return review_num

꿀스틱 상품 목록 진입

In [512]:
keyword = '꿀스틱'
path = r"C:/chromedriver.exe"
url_goodsno = f'https://search.shopping.naver.com/search/all?frm=NVSCPRO&origQuery=%EA%BF%80%EC%8A%A4%ED%8B%B1&pagingIndex=8&pagingSize=40&productSet=total&query=%EA%BF%80%EC%8A%A4%ED%8B%B1&sort=review&timestamp=&viewType=list'
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.get(url_goodsno)

상품 url 리스트로 수집

In [513]:
url_list = []
for n in range(1, 41):
    store_url = driver.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/div[2]/div/div[{n}]/div/div/div[3]/div[1]/a[1]').get_attribute('href')
    if store_url[:28] == 'https://smartstore.naver.com':
        product_url = driver.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/div[2]/div/div[{n}]/div/div/div[2]/div[1]/a').get_attribute('href')
        url_list.append(product_url)
        print(n, '번째 상품 수집대상 추가')
    else:
        print(n, '번째 상품 수집대상 제외', '-'*30)

1 번째 상품 수집대상 추가
2 번째 상품 수집대상 제외 ------------------------------
3 번째 상품 수집대상 추가
4 번째 상품 수집대상 추가
5 번째 상품 수집대상 추가
6 번째 상품 수집대상 추가
7 번째 상품 수집대상 추가
8 번째 상품 수집대상 추가
9 번째 상품 수집대상 추가
10 번째 상품 수집대상 추가
11 번째 상품 수집대상 제외 ------------------------------
12 번째 상품 수집대상 제외 ------------------------------
13 번째 상품 수집대상 제외 ------------------------------
14 번째 상품 수집대상 추가
15 번째 상품 수집대상 추가
16 번째 상품 수집대상 추가
17 번째 상품 수집대상 제외 ------------------------------
18 번째 상품 수집대상 추가
19 번째 상품 수집대상 추가
20 번째 상품 수집대상 추가
21 번째 상품 수집대상 제외 ------------------------------
22 번째 상품 수집대상 제외 ------------------------------
23 번째 상품 수집대상 추가
24 번째 상품 수집대상 제외 ------------------------------
25 번째 상품 수집대상 추가
26 번째 상품 수집대상 제외 ------------------------------
27 번째 상품 수집대상 추가
28 번째 상품 수집대상 추가
29 번째 상품 수집대상 제외 ------------------------------
30 번째 상품 수집대상 제외 ------------------------------
31 번째 상품 수집대상 제외 ------------------------------
32 번째 상품 수집대상 제외 ------------------------------
33 번째 상품 수집대상 제외 ------------------------------
34 번째 상품 수집대상 

In [514]:
len(url_list)

23

리뷰 클릭

In [588]:
driver.get(url_list[18])
time.sleep(2)
driver.find_elements(By.CLASS_NAME, '_1k5R-niA93')[1].click()

In [589]:
review_list = []
review_num = review_count()
page_review = page_count(review_num, 20)
#name
product = driver.find_element(By.CLASS_NAME, f'_1eddO7u4UC').text
#price
price = driver.find_element(By.CLASS_NAME, f'_1LY7DqCnwR').text
#rating
product_rating = driver.find_elements(By.CLASS_NAME, f'_2pgHN-ntx6')[1].text[:3]
for loop in range(1, page_review+1):
    for k in range(20):
        try:
            review_box = driver.find_elements(By.CLASS_NAME, f'_1yIGHygFbx')[k]
                                                        
            id = review_box.find_elements(By.CLASS_NAME, '_3QDEeS6NLn')[0].text
            rating = review_box.find_element(By.CLASS_NAME, '_15NU42F3kT').text
            date = review_box.find_elements(By.CLASS_NAME, '_3QDEeS6NLn')[1].text
            product_select = re.findall(r'.+\n', review_box.find_element(By.CLASS_NAME, '_14FigHP3K8').text)[0]
            review_option = re.findall(r'유통기한.+', review_box.find_element(By.CLASS_NAME, '_14FigHP3K8').text)[0]
            review_text = review_box.find_element(By.CLASS_NAME, 'YEtwtZFLDz').text
            review_list.append([product, price, product_rating, id, rating, date, product_select, review_option, review_text])
        except:continue
    if loop != page_review:
        
        try:
            driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div[2]/div/div[3]/div[7]/div/div[3]/div[2]/div/div/a[11]')
            if loop % 10 == 0:
                driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
            else:
                driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a:nth-child({(loop%10)+2})').click()
        except:
            driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
        time.sleep(0.5)
    print('페이지 진행 상황 :', loop, '/', page_review, '완료', '-'*30)


페이지 진행 상황 : 1 / 3 완료 ------------------------------
페이지 진행 상황 : 2 / 3 완료 ------------------------------
페이지 진행 상황 : 3 / 3 완료 ------------------------------


In [590]:
review_list

[]

In [587]:
# 3_6, 3_12, 3_15, 3_22없음.제품명   가격   평균평점   아이디   평점   날짜   종류   요약   리뷰
# 4_5, 4_7없음
# 5_10, 5_11 없음
# 6_6, 6_10, 6_11, 6_14, 6_15, 6_16, 6_17,  없음
# 7_0, 7_4, 7_7, 7_8, 7_11, 7_17,7_18  없음
# 8_7, 8_10 없음
df_8_17 = pd.DataFrame(review_list, columns=['제품명', '가격', '평균평점', '아이디', '평점', '날짜', '종류', '요약', '리뷰'])
len(df_8_17)

50

In [591]:
naver_shopping_8 = pd.concat([df_8_1, df_8_2, df_8_3, df_8_4, df_8_5, df_8_6, 
                              df_8_8, df_8_9, df_8_9, df_8_11, df_8_12, df_8_13, df_8_14,
                              df_8_15, df_8_16, df_8_17])
len(naver_shopping_8)

834

In [592]:
naver_shopping_8.to_csv('naver_shopping_8.csv')

In [599]:
df = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_4.csv')

2620

In [600]:
df1 = pd.read_excel(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_4.xlsx')
len(df1)

2620

In [ ]:
df = pd.DataFrame(review_list, columns=['product', 'price', 'product_rating', 'id', 'rating', 'date', 'product_select', 'review_option', 'review_text'])

In [15]:
df1 = pd.read_csv(r'C:\Users\NT550\asac\SIH\KYJ\honey_stick\data\navershopping_data_page1.csv') 
df1.columns = ['Unnamed: 0','제품명', '가격', '평균평점', '아이디', '평점', '날짜', '종류', '요약', '리뷰']

df1.head()

,Unnamed: 0,제품명,가격,평균평점,아이디,평점,날짜,종류,요약,리뷰
0,0,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,miri*******,5,23.08.16.,30포 선택1: 아카시아 / 30포 선택2: 아카시아 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기재구매먹기 편하고 품질도 좋아요. 받는 손님도 좋아 하시고 구매등급도 따로...
1,1,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,dltm*****,5,23.07.28.,30포 선택1: 밤 / 30포 선택2: 밤 / 30포 선택3: 밤\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,간편하게 하루한포씩먹게되요~냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요!!밤꿀이...
2,2,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,yjdr****,5,23.08.06.,30포 선택1: 올인원 / 30포 선택2: 올인원 / 30포 선택3: 아카시아\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,홍삼이 여자몸에 안좋다고해서.. 그동안 안먹다가 꿀 스틱 보고 주문했습니다\n당떨어...
3,3,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,2133***,5,23.07.13.,30포 선택1: 밤 / 30포 선택2: 야생화 / 30포 선택3: 올인원\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,"BEST꿀이 피로 회복, 기관지에 좋다고 하는데 사실 어느정도가 적당한지, 너무 많..."
4,4,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,klys****,5,23.08.12.,30포 선택1: 야생화 / 30포 선택2: 밤 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기진짜 꿀이예요. 예전 시골에서 먹던 꿀맛과 같아요. 술먹고 먹어도 숙취에도...


In [2]:
df1 = pd.read_csv(r'C:\Users\NT550\asac\SIH\KYJ\honey_stick\data\navershopping_data_page1.csv') 
df2 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_2.csv')
df3 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_3.csv')
df4 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_4.csv')
df5 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_5.csv')
df6 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_6.csv')
df7 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_7.csv')
df8 = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_8.csv')


In [5]:
df1 = pd.read_csv(r'C:\Users\NT550\asac\SIH\KYJ\honey_stick\data\navershopping_data_page1.csv') 
df1.columns = ['Unnamed: 0','제품명', '가격', '평균평점', '아이디', '평점', '날짜', '종류', '요약', '리뷰']

df1.shape

(42784, 10)

In [32]:
#df2 = df2.drop(['Unnamed: 0','평'], axis=1)
#df2.columns = ['Unnamed: 0','제품명', '가격', '평균평점', '아이디', '평점', '날짜', '종류', '요약', '리뷰']
df4 = df4.drop('Unnamed: 0.1', axis=1)

In [33]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)
print(df8.shape)

(42784, 10)
(4552, 10)
(3015, 10)
(2620, 10)
(1149, 10)
(1199, 10)
(988, 10)
(834, 10)


In [35]:
#naver_shopping = pd.concat([df1,df2, df3, df4, df5, df6, df7, df8])
print(len(naver_shopping))
naver_shopping = naver_shopping.drop('Unnamed: 0', axis=1)
naver_shopping.head() 

57141


,제품명,가격,평균평점,아이디,평점,날짜,종류,요약,리뷰
0,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,miri*******,5.0,23.08.16.,30포 선택1: 아카시아 / 30포 선택2: 아카시아 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기재구매먹기 편하고 품질도 좋아요. 받는 손님도 좋아 하시고 구매등급도 따로...
1,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,dltm*****,5.0,23.07.28.,30포 선택1: 밤 / 30포 선택2: 밤 / 30포 선택3: 밤\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,간편하게 하루한포씩먹게되요~냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요!!밤꿀이...
2,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,yjdr****,5.0,23.08.06.,30포 선택1: 올인원 / 30포 선택2: 올인원 / 30포 선택3: 아카시아\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,홍삼이 여자몸에 안좋다고해서.. 그동안 안먹다가 꿀 스틱 보고 주문했습니다\n당떨어...
3,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,2133***,5.0,23.07.13.,30포 선택1: 밤 / 30포 선택2: 야생화 / 30포 선택3: 올인원\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,"BEST꿀이 피로 회복, 기관지에 좋다고 하는데 사실 어느정도가 적당한지, 너무 많..."
4,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,klys****,5.0,23.08.12.,30포 선택1: 야생화 / 30포 선택2: 밤 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기진짜 꿀이예요. 예전 시골에서 먹던 꿀맛과 같아요. 술먹고 먹어도 숙취에도...


In [36]:
naver_shopping.to_csv('naver_shopping_review.csv')

In [39]:
df = pd.read_csv(r'C:\Users\NT550\asac\SIH\SIH\code2\result\naver_shopping_review.csv')
df.head()

,Unnamed: 0,제품명,가격,평균평점,아이디,평점,날짜,종류,요약,리뷰
0,0,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,miri*******,5.0,23.08.16.,30포 선택1: 아카시아 / 30포 선택2: 아카시아 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기재구매먹기 편하고 품질도 좋아요. 받는 손님도 좋아 하시고 구매등급도 따로...
1,1,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,dltm*****,5.0,23.07.28.,30포 선택1: 밤 / 30포 선택2: 밤 / 30포 선택3: 밤\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,간편하게 하루한포씩먹게되요~냉장이나 냉동으로 해서 시원하게 먹으니 맛잇어요!!밤꿀이...
2,2,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,yjdr****,5.0,23.08.06.,30포 선택1: 올인원 / 30포 선택2: 올인원 / 30포 선택3: 아카시아\n,유통기한꽤 남았어요포장꼼꼼해요편리편리해요,홍삼이 여자몸에 안좋다고해서.. 그동안 안먹다가 꿀 스틱 보고 주문했습니다\n당떨어...
3,3,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,2133***,5.0,23.07.13.,30포 선택1: 밤 / 30포 선택2: 야생화 / 30포 선택3: 올인원\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,"BEST꿀이 피로 회복, 기관지에 좋다고 하는데 사실 어느정도가 적당한지, 너무 많..."
4,4,허니바이허니 천연 벌꿀 스틱 90포입 국산 아카시아 5종 선물 세트 [원산지:국산]...,"72,000",4.9,klys****,5.0,23.08.12.,30포 선택1: 야생화 / 30포 선택2: 밤 / 30포 선택3: 아카시아\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기진짜 꿀이예요. 예전 시골에서 먹던 꿀맛과 같아요. 술먹고 먹어도 숙취에도...


In [21]:
naver_shopping.head()


,Unnamed: 0.1,제품명,가격,평균평점,아이디,평점,날짜,종류,요약,리뷰,Unnamed: 0,평
0,0.0,호주산 Bee2 벌꿀 허니스틱 250포 대용량 (마누카꿀 MGO)+호주 마약커피 [...,"400,000",4.9,ami0***,5.0,23.08.10.,제품선택: 마누카꿀 MGO300+12g x 250개+커피\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,아주 찐해요!! 연하게 타먹어야겠어요,NaN,NaN
1,1.0,호주산 Bee2 벌꿀 허니스틱 250포 대용량 (마누카꿀 MGO)+호주 마약커피 [...,"400,000",4.9,cns6***,3.0,23.08.17.,제품선택: 벌꿀스틱 (마누카꿀 MGO 0) 250개+커피\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,맛도좋고 건강해질까 꾸준히먹고있어요,NaN,NaN
2,2.0,호주산 Bee2 벌꿀 허니스틱 250포 대용량 (마누카꿀 MGO)+호주 마약커피 [...,"400,000",4.9,hdg6***,5.0,23.08.11.,제품선택: 벌꿀스틱 (마누카꿀 MGO 0) 250개+커피\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,꿀 최고입니다.수량도250개로 넉넉하게먹을양입니다,NaN,NaN
3,3.0,호주산 Bee2 벌꿀 허니스틱 250포 대용량 (마누카꿀 MGO)+호주 마약커피 [...,"400,000",4.9,ha****,5.0,23.07.26.,제품선택: 마누카꿀 MGO300+12g x 250개+커피\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,재구매꾸준히 먹고 있어요.,NaN,NaN
4,4.0,호주산 Bee2 벌꿀 허니스틱 250포 대용량 (마누카꿀 MGO)+호주 마약커피 [...,"400,000",4.9,ru****,5.0,23.07.15.,제품선택: 마누카꿀 MGO300+12g x 250개+커피\n,유통기한아주 넉넉해요포장꼼꼼해요편리편리해요,한달사용기첫 리뷰 내용이 잘못 돼서 다시 남김니다. (택배로 뉘어와서 착시였던 건지...,NaN,NaN


페이지별 제품 리뷰 다 수집

In [10]:
review_list = []
for n in range(1, 41):
    driver.get(url_goodsno)
    time.sleep(3)
    store_url = driver.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/div[2]/div/div[{n}]/div/div/div[3]/div[1]/a[1]').get_attribute('href')
    #셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
    prev_height = driver.execute_script("return document. body.scrollHeight")
    import time
    while True:
        #첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        #시간대기
        time.sleep(5)

        #현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")
        time.sleep(3)
        #현재높이와 끝의 높이가 끝이면 탈출
        if current_height == prev_height:
            break
        #업데이트해줘서 끝낼 수 있도록
        time.sleep(1)
        prev_height == current_height
    if store_url[:28] == 'https://smartstore.naver.com':
        product_url = driver.find_element(By.XPATH, f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/div[2]/div/div[{n}]/div/div/div[2]/div[1]/a').get_attribute('href')
        driver.get(product_url)
        time.sleep(2)
        driver.find_elements(By.CLASS_NAME, '_1k5R-niA93')[1].click()
        review_num = review_count()
        page_review = page_count(review_num, 20)
        #name
        product = driver.find_element(By.CLASS_NAME, f'_1eddO7u4UC').text
        #price
        price = driver.find_element(By.CLASS_NAME, f'_1LY7DqCnwR').text
        #rating
        product_rating = driver.find_elements(By.CLASS_NAME, f'_2pgHN-ntx6')[1].text[:3]
        for loop in range(1, page_review+1):
            for k in range(20):
                try:
                    review_box = driver.find_elements(By.CLASS_NAME, f'_1yIGHygFbx')[k]
                                                                
                    id = review_box.find_elements(By.CLASS_NAME, '_3QDEeS6NLn')[0].text
                    rating = review_box.find_element(By.CLASS_NAME, '_15NU42F3kT').text
                    date = review_box.find_elements(By.CLASS_NAME, '_3QDEeS6NLn')[1].text
                    product_select = re.findall(r'.+\n', review_box.find_element(By.CLASS_NAME, '_14FigHP3K8').text)[0]
                    review_option = re.findall(r'유통기한.+', review_box.find_element(By.CLASS_NAME, '_14FigHP3K8').text)[0]
                    review_text = review_box.find_element(By.CLASS_NAME, 'YEtwtZFLDz').text
                    review_list.append([product, price, product_rating, id, rating, date, product_select, review_option, review_text])
                except:continue
            if loop != page_review:
                
                try:
                    driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div[2]/div/div[3]/div[7]/div/div[3]/div[2]/div/div/a[11]')
                    if loop % 10 == 0:
                        driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
                    else:
                        driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a:nth-child({(loop%10)+2})').click()
                except:
                    driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
                time.sleep(0.5)
            print(n, '번째 상품', loop, '/', page_review, '페이지 완료', '-'*30)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.96)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF7CDFE4A62+57106]
	(No symbol) [0x00007FF7CDF5CF52]
	(No symbol) [0x00007FF7CDE2E2CB]
	(No symbol) [0x00007FF7CDE0DEB9]
	(No symbol) [0x00007FF7CDE8AE07]
	(No symbol) [0x00007FF7CDE9DBEF]
	(No symbol) [0x00007FF7CDE86793]
	(No symbol) [0x00007FF7CDE5CE81]
	(No symbol) [0x00007FF7CDE5E064]
	GetHandleVerifier [0x00007FF7CE294222+2873042]
	GetHandleVerifier [0x00007FF7CE2E6590+3209792]
	GetHandleVerifier [0x00007FF7CE2DF3AF+3180639]
	GetHandleVerifier [0x00007FF7CE075F25+652245]
	(No symbol) [0x00007FF7CDF68618]
	(No symbol) [0x00007FF7CDF647C4]
	(No symbol) [0x00007FF7CDF648BC]
	(No symbol) [0x00007FF7CDF54C33]
	BaseThreadInitThunk [0x00007FFAC97626AD+29]
	RtlUserThreadStart [0x00007FFACB5AAA68+40]


In [7]:
df = pd.DataFrame(review_list, columns=['product', 'price', 'product_rating', 'id', 'rating', 'date', 'product_select', 'review_option', 'review_text'])

8933